In [12]:
from pynq import Overlay
from pynq import Xlnk
import math
import numpy as np
import pynq.lib.dma
import time

FFT_MAX_NUM_PTS      = 8192
FFT_NUM_PTS_MASK     = 0x0000001F # Bits [4:0]
FFT_NUM_PTS_SHIFT    = 0
FFT_FWD_INV_MASK     = 0x00000100 # Bit 8
FFT_FWD_INV_SHIFT    = 8
FFT_SCALE_SCH_MASK   = 0x007FFE00 # Bits [22:9]
FFT_SCALE_SCH_SHIFT  = 9
FFT_INVERSE = 0
FFT_FORWARD = 1

# n = 64
n = 1024 //2

ol = Overlay("./bits/pynq_fft/pynq_fft.bit", download=True)
print("Download ok.")
# dir(ol)
gpio = ol.axi_gpio_0

def set_values(scale, fwd_inv, num_pts):
    reg = 0;
    reg = (scale << FFT_SCALE_SCH_SHIFT) & FFT_SCALE_SCH_MASK;
    reg |= (fwd_inv << FFT_FWD_INV_SHIFT) & FFT_FWD_INV_MASK;
    reg |= (int(math.log2(num_pts)) << FFT_NUM_PTS_SHIFT) & FFT_NUM_PTS_MASK;
    gpio.write(0, reg)

xlnk = Xlnk()
# in_buf = xlnk.cma_array(shape=(n, ), dtype=np.int16)
# out_buf = xlnk.cma_array(shape=(n, ), dtype=np.int16)
in_buf = xlnk.cma_array(shape=(n, ), dtype=np.float)
out_buf = xlnk.cma_array(shape=(n, ), dtype=np.float)

# help(in_buf)

for i in range(n):
    # in_buf[i] = 1 / (i % 0xF + 1)
    in_buf[i] = int(math.sin((i + 30) * 10 / 180) * 0xFF) - (0xFF // 2)
    # in_buf[i] = int(math.sin((i + 30) * 20 / 90) * 0xFF) - (0xFF // 2)
    # in_buf[i] = 0

# print(in_buf[i])

# np.copyto(out_buf, np.zeros(shape=(n, ), dtype=np.int16))
np.copyto(out_buf, np.zeros(shape=(n, ), dtype=np.float))

set_values(0x2AB, FFT_FORWARD, n)
# set_values(0x01, FFT_FORWARD, n)

time.sleep(0.3)

dma = ol.dma_fft
dma.sendchannel.transfer(in_buf)
dma.recvchannel.transfer(out_buf)
dma.sendchannel.wait()
time.sleep(0.3)
dma.recvchannel.wait()

# print(in_buf, out_buf)

data_in, data_out = np.copy(in_buf), np.copy(out_buf)
dma.sendchannel.stop()
dma.recvchannel.stop()
in_buf.close()
out_buf.close()

print("in:", data_in.shape, "out:", data_out.shape)

with open("./test/in.dat", "w") as f:
    for i in data_in:
        f.write("%s\n" % i)
with open("./test/out.dat", "w") as f:
    for i in data_out:
        f.write("%s\n" % i)

print("All Done.")

Download ok.
in: (512,) out: (512,)
All Done.
